# Example_module_wellsKNN_v1

Justin Gosses May 2019
### Uses these modules of Predictatops:
1. WellsKNN

This notebook is modeled after the code in wellsKNN_runner.py in predictatops source folder

First, it should be noted that Predictatops isn't on Pypy, so we're going go to be loading the Python modules from a local directory and this notebook is running from the demo folder in the same Predictatops directory. If you run it from elsewhere, you'll have to change the paths.



In [1]:
ls

Example_firstSteps_modules_fetchdata_configuration_checkdata.ipynb
Example_junk_post_balance.ipynb
Example_module_all_runner_temp_v1b.ipynb
Example_module_balance_v1.ipynb
Example_module_predictionclasses_v1.ipynb
Example_module_trainclasses_v2.ipynb
Example_module_wellsKNN_v1.ipynb
README.md
current_errors_TopMcMr_20181006.png
mannville_demo_data/
mannville_demo_data.zip


In [2]:
cd ..

/Users/justingosses/Code/predictatops


In [3]:
cd predictatops/

/Users/justingosses/Code/predictatops/predictatops


In [4]:
ls

__init__.py                       load_runner.py
__pycache__/                      main.py
all_runner.py                     map.py
balance.py                        plot.py
balance_runner.py                 plot_runner.py
checkdata.py                      predictionclasses.py
checkdata_runner.py               predictionclasses_runner.py
cli.py                            registry.txt
configurationplusfiles.py         split.py
configurationplusfiles_runner.py  split_runner.py
dask-worker-space/                trainclasses.py
features.py                       trainclasses_runner.py
features_runner.py                wellsKNN.py
fetch_demo_data.py                wellsKNN_runner.py
load.py


Now we should be in the Predictatops source file and see various .py files

In [5]:
################ imports ###################
import pandas as pd
################ import from other python files in this package ###################
from wellsKNN import *
from configurationplusfiles_runner import input_data_inst, config, output_data_inst

head of picks df =      SitID  HorID      Pick  Quality
0  102496   1000       321        1
1  102496   2000                 -1
2  102496   3000                 -1
3  102496   4000                 -1
4  102496   5000       438        2
making base folder for results in: ../results_higherBlanceAndLowerWindows_20190517
base_path directory already exists, ../results_higherBlanceAndLowerWindows_20190517  so not creating it again. This may or may not be what you intended, so just flagging it.
directory  checkData  already exists so not making it again in make_all_directories function of configurationplusfiles.py
directory  load  already exists so not making it again in make_all_directories function of configurationplusfiles.py
directory  split  already exists so not making it again in make_all_directories function of configurationplusfiles.py
directory  wellsKNN  already exists so not making it again in make_all_directories function of configurationplusfiles.py
directory  features  already 

################ Load wells df saved from running split functions ###################
################ This file is wells loaded with no features but does have a column for train test split ##################

In [6]:
##### path to input file ######
split_dir = output_data_inst.base_path_for_all_results+ "/" + output_data_inst.path_split
split_file = output_data_inst.split_results_wells_df
ending = output_data_inst.default_results_file_format
full_path_to_split_results = split_dir+"/"+split_file+ending
##### loading wells df created in split ######
wells_df_from_split  = pd.read_hdf(full_path_to_split_results)
print("wells_df_from_split.columns",wells_df_from_split.columns)

wells_df_from_split.columns Index(['CALI', 'COND', 'DELT', 'DEPT', 'DPHI', 'DT', 'GR', 'ILD', 'ILM',
       'NPHI', 'PHID', 'RHOB', 'SFL', 'SFLU', 'SN', 'SP', 'UWI',
       'trainOrTest'],
      dtype='object')


### Load all df for picks, pick dictionary, wells, and coordinate data.
### Paths for loading these come from input_data class in configurationplusfiles.py

In [7]:
################ Load all df for picks, pick dictionary, wells, and coordinate data. ###################
################ Paths for loading these come from input_data class in configurationplusfiles.py ###################
picks, picks_dic, wells, gis  = get_data_for_wellsKNN(input_data_inst)

### Target is the name of the top we're trying to predict
### target_base is the name of the top under if we're using for guidance

In [8]:
################ Target is the name of the top we're trying to predict ###################
################ target_base is the name of the top under if we're using for guidance ###################
target, target_base = getTopsForWantedPickNames(config)

Generates two dataframes. Each with the pick depths of only a given top.

In [9]:
################ Generates two dataframes. Each with the pick depths of only a given top. ###################
picks_targetTop, picks_targetBase = findAllPicksForTops(picks, target, target_base, config.HorID_name_col_in_picks_df)


Merges many of the dataframes together based on config.siteID_col_in_picks_df 

In [10]:
################ Merges many of the dataframes together based on config.siteID_col_in_picks_df ###################
df_new = mergeDataframes(wells,picks_targetTop,config.siteID_col_in_picks_df,picks_targetBase, gis)

print(df_new.head())

    SitID    UWI (AGS)_x                UWI_x  HorID_x Pick_x  Quality_x  \
0  102496  0674010812000  00/12-08-067-01W4/0    14000    561          1   
1  102497  0674020807000  00/07-08-067-02W4/0    14000  604.5          1   
2  102498  0674021109000  00/09-11-067-02W4/0    14000    564          1   
3  102500  0674022910000  00/10-29-067-02W4/0    14000  636.5          1   
4  102501  0674023406000  00/06-34-067-02W4/0    14000    613          1   

     UWI (AGS)_y                UWI_y  HorID_y Pick_y  Quality_y        lat  \
0  0674010812000  00/12-08-067-01W4/0    13000    475          3  54.785907   
1  0674020807000  00/07-08-067-02W4/0    13000    515          3  54.782284   
2  0674021109000  00/09-11-067-02W4/0    13000    480          3  54.785892   
3  0674022910000  00/10-29-067-02W4/0    13000    549          3  54.829624   
4  0674023406000  00/06-34-067-02W4/0    13000    529          2  54.840471   

          lng      UWI (AGS)                  UWI  HorID Pick  Quali

Let's limit the GIS and pick information dataframe to only the wells we're going to use and exclude any we're not going to use for not having the needed tops and curves. We'll have to change the format of the UWU col slightly to do this

In [11]:
################ Let's limit the GIS and pick information dataframe to only the wells we're going to use and ###################
################ exclude any we're not going to use for not having the needed tops and curves. ###################
################ We'll have to change the format of the UWU col slightly to do this ###################

wellsLoaded_list = createListOfWellNamesLoadedSplit(wells_df_from_split,config)

examples of what the well format looks like in wellsLoaded_list ['00-11-21-077-07W4-0.LAS' '00-07-26-074-05W4-0.LAS'
 '00-12-19-075-09W4-0.LAS']


In [12]:
new_wells_loaded_list_inUWIstyle = findListOfConvertedUWInamesForWellsLoaded(wells_df_from_split,config.UWI,input_data_inst.well_format)
print("new_wells_loaded_list_inUWIstyle[5:8]",new_wells_loaded_list_inUWIstyle[5:8])


new_wells_loaded_list_inUWIstyle[5:8] ['00/11-11-080-19W4/0', '00/06-25-082-09W4/0', '00/06-31-067-15W4/0']


In [13]:
################ This next step will reduce the total number of wells to just the ones that have at least the tops and curves we need ################ 

df_reduced = reduced_df_from_split_plus_more(df_new,new_wells_loaded_list_inUWIstyle,config.UWI)

len(df_new_allWells) 2193


In [14]:
################ We'll now find nearest N neighbors using a kdtree approach ################ 
tree, dist, ind = kdtree(df_reduced, input_data_inst.gis_lat_col, input_data_inst.gis_long_col, config.kdtree_leaf, config.kdtree_k)


UWIs_Geog = makeKNearNeighObj(df_reduced,config.UWI,input_data_inst.gis_lat_col, input_data_inst.gis_long_col,dist,ind,config.kdtree_k)
print("UWIs_Geog.head()",UWIs_Geog.head())

UWIs_Geog.head()                    UWI        lat         lng  \
0  00/12-08-067-01W4/0  54.785907 -110.129320   
1  00/07-08-067-02W4/0  54.782284 -110.269446   
2  00/09-11-067-02W4/0  54.785892 -110.186851   
3  00/10-29-067-02W4/0  54.829624 -110.269422   
4  00/06-34-067-02W4/0  54.840471 -110.224832   

                                       Neighbors_Obj  
0  [{'UWI': '00/09-11-067-02W4/0', 'neighbor': 1,...  
1  [{'UWI': '00/10-29-067-02W4/0', 'neighbor': 1,...  
2  [{'UWI': '00/12-08-067-01W4/0', 'neighbor': 1,...  
3  [{'UWI': '00/10-08-068-02W4/0', 'neighbor': 1,...  
4  [{'UWI': '00/10-29-067-02W4/0', 'neighbor': 1,...  


In [16]:
################# Now to clean the dataframe, rename columns, and merge with the new dataframe that has the neighbors! ##################
df_new_cleaned = cleanRenameDF(df_reduced,target, target_base,config,input_data_inst)

print("type df_new_cleaned",type(df_new_cleaned))
#print(df_new_cleaned)
print("df_new_cleaned.head()",df_new_cleaned.head())

type df_new_cleaned <class 'pandas.core.frame.DataFrame'>
df_new_cleaned.head()     SitID        lat         lng                  UWI  TopHelper_HorID  \
0  102496  54.785907 -110.129320  00/12-08-067-01W4/0            14000   
1  102497  54.782284 -110.269446  00/07-08-067-02W4/0            14000   
2  102498  54.785892 -110.186851  00/09-11-067-02W4/0            14000   
3  102500  54.829624 -110.269422  00/10-29-067-02W4/0            14000   
4  102501  54.840471 -110.224832  00/06-34-067-02W4/0            14000   

   TopTarget_HorID TopHelper_DEPTH TopTarget_DEPTH  TopHelper_HorID_Qual  \
0            13000             561             475                     1   
1            13000           604.5             515                     1   
2            13000             564             480                     1   
3            13000           636.5             549                     1   
4            13000             613             529                     1   

   TopTarget_Qual 

In [17]:
df_new_cleaned_plus_nn = mergeCleanedAndUWIGeog_dfs(df_new_cleaned,UWIs_Geog)

print("This is how to access the data on nearest neigbhors in the 'Neighbors_Obj' column - df_new_cleaned_plus_nn.loc[1:1,'Neighbors_Obj'][1][1] =",df_new_cleaned_plus_nn.loc[1:1,'Neighbors_Obj'][1][1])

print("an example of neighbors is : df_new_cleaned_plus_nn.loc[1:2,'Neighbors_Obj'][2] which is = ",df_new_cleaned_plus_nn.loc[1:2,'Neighbors_Obj'][2])


This is how to access the data on nearest neigbhors in the 'Neighbors_Obj' column - df_new_cleaned_plus_nn.loc[1:1,'Neighbors_Obj'][1][1] = {'UWI': '00/06-34-067-02W4/0', 'neighbor': 2, 'distance': 0.07332213830078355}
an example of neighbors is : df_new_cleaned_plus_nn.loc[1:2,'Neighbors_Obj'][2] which is =  [{'UWI': '00/12-08-067-01W4/0', 'neighbor': 1, 'distance': 0.05753100195547899}, {'UWI': '00/06-34-067-02W4/0', 'neighbor': 2, 'distance': 0.06649378619088428}, {'UWI': '00/07-08-067-02W4/0', 'neighbor': 3, 'distance': 0.08267376663125782}, {'UWI': '00/09-11-068-02W4/0', 'neighbor': 4, 'distance': 0.08728400165551212}, {'UWI': '00/10-29-067-02W4/0', 'neighbor': 5, 'distance': 0.09343691917544036}, {'UWI': '00/11-08-068-01W4/0', 'neighbor': 6, 'distance': 0.10808166919974113}, {'UWI': '00/10-08-068-02W4/0', 'neighbor': 7, 'distance': 0.12013967631468725}]


In [18]:
################# 6 and 7 in the functions below are the 6th and 7th columns... should replace with something more obvious!!!!! ##################
df_new_cleaned_plus_nn_temp = broadcastFuncForFindNearestNPickDepth(df_new_cleaned_plus_nn, 7, config.NN1_topTarget_DEPTH, config.UWI)
df_new2 = broadcastFuncForFindNearestNPickDepth(df_new_cleaned_plus_nn_temp, 6, config.NN1_TopHelper_DEPTH, config.UWI)


df_new3 = useThicknessOfNeighborsToEst(df_new2)

print("printing first row of df_new2 in wellsKNN_runner.py",df_new3[0:1])


0   NaN
Name: NN1_topTarget_DEPTH, dtype: float64


/Users/justingosses/Code/predictatops/predictatops/wellsKNN.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df.iloc[eachRow:eachRowP1][newPickColName] = pick_depth


0   NaN
Name: NN1_TopHelper_DEPTH, dtype: float64
printing first row of df_new2 in wellsKNN_runner.py     SitID        lat        lng                  UWI  TopHelper_HorID  \
0  102496  54.785907 -110.12932  00/12-08-067-01W4/0            14000   

   TopTarget_HorID  TopHelper_DEPTH  TopTarget_DEPTH  TopHelper_HorID_Qual  \
0            13000            561.0            475.0                     1   

   TopTarget_Qual                                      Neighbors_Obj  \
0               3  [{'UWI': '00/09-11-067-02W4/0', 'neighbor': 1,...   

   NN1_topTarget_DEPTH  NN1_TopHelper_DEPTH  NN1_thickness  \
0                544.0                630.0           86.0   

   topTarget_Depth_predBy_NN1thick  
0                            475.0  


In [19]:
df_new3 = create_diff_Top_Depth_Real_v_predBy_NN1thick(df_new3)

In [20]:
###### df_new3 has both train and test. That is find for finding neighbors for test wells, but
###### for train wells we should only be including information from other train wells. 
###### it would not be fair to include test well neihbor information in training data!!!
###### We'll redo the steps above now but just for training wells. 
###### Firs though, we'll take what we've build and slice off just the test wells into a new dataframe.

df_completed_test = getTestRowsOnlyPicksDF(df_new3,wells_df_from_split,config,"test",input_data_inst.well_format)

len(UWI_test) 321
UWI_test[0] AA/11-33-094-06W4/0
df[config.UWI][0] 00/12-08-067-01W4/0
len(df_new) 321


In [21]:
#### okay now lets run the whole thing with just the train wells

df_train = onlyWellsInTestPortion(wells_df_from_split,"train")

df_nearly_completed_train = fullWellsKNN(df_train,input_data_inst,config)
print("len(df_nearly_completed_train)",len(df_nearly_completed_train))
df_completed_train = getTestRowsOnlyPicksDF(df_nearly_completed_train,df_train,config,"train",input_data_inst.well_format)
print("len(df_completed_test) =",len(df_completed_test))
print("len(df_completed_train)",len(df_completed_train))

df_completed_trainAndTest = pd.concat([df_completed_train, df_completed_test])
print("length df_completed_trainAndTest = ",len(df_completed_trainAndTest))

len(df_just_trainOrMaybeTest) 1041163
len(df_just_trainOrMaybeTest['UWI'].unique()) 1280
len(wells_df_from_split) 1041163
len(df_new ) 2193
examples of what the well format looks like in wellsLoaded_list ['00-11-21-077-07W4-0.LAS' '00-07-26-074-05W4-0.LAS'
 '00-12-19-075-09W4-0.LAS']
new_wells_loaded_list_inUWIstyle[5:8] ['00/11-11-080-19W4/0', '00/06-25-082-09W4/0', '00/06-31-067-15W4/0']
len(new_wells_loaded_list_inUWIstyle ) 1280
len(df_new_allWells) 2193
len(df_reduced) 2193
UWIs_Geog.head()                    UWI        lat         lng  \
0  00/12-08-067-01W4/0  54.785907 -110.129320   
1  00/07-08-067-02W4/0  54.782284 -110.269446   
2  00/09-11-067-02W4/0  54.785892 -110.186851   
3  00/10-29-067-02W4/0  54.829624 -110.269422   
4  00/06-34-067-02W4/0  54.840471 -110.224832   

                                       Neighbors_Obj  
0  [{'UWI': '00/09-11-067-02W4/0', 'neighbor': 1,...  
1  [{'UWI': '00/10-29-067-02W4/0', 'neighbor': 1,...  
2  [{'UWI': '00/12-08-067-01W4/0', 'nei

/Users/justingosses/Code/predictatops/predictatops/wellsKNN.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df.iloc[eachRow:eachRowP1][newPickColName] = pick_depth


0   NaN
Name: NN1_TopHelper_DEPTH, dtype: float64
len(df_new3) 2193
printing first row of df_new2 in wellsKNN_runner.py     SitID        lat        lng                  UWI  TopHelper_HorID  \
0  102496  54.785907 -110.12932  00/12-08-067-01W4/0            14000   

   TopTarget_HorID  TopHelper_DEPTH  TopTarget_DEPTH  TopHelper_HorID_Qual  \
0            13000            561.0            475.0                     1   

   TopTarget_Qual                                      Neighbors_Obj  \
0               3  [{'UWI': '00/09-11-067-02W4/0', 'neighbor': 1,...   

   NN1_topTarget_DEPTH  NN1_TopHelper_DEPTH  NN1_thickness  \
0                544.0                630.0           86.0   

   topTarget_Depth_predBy_NN1thick  
0                            475.0  
len(df_new4) 2193
len(df_nearly_completed_train) 2193
len(UWI_test) 1280
UWI_test[0] 00/10-32-080-20W4/0
df[config.UWI][0] 00/12-08-067-01W4/0
len(df_new) 1280
len(df_completed_test) = 321
len(df_completed_train) 1280
length df_comp

### Examining the resulting dataframe:

In [22]:
df_completed_trainAndTest.head()

,SitID,lat,lng,UWI,TopHelper_HorID,TopTarget_HorID,TopHelper_DEPTH,TopTarget_DEPTH,TopHelper_HorID_Qual,TopTarget_Qual,Neighbors_Obj,NN1_topTarget_DEPTH,NN1_TopHelper_DEPTH,NN1_thickness,topTarget_Depth_predBy_NN1thick,diff_Top_Depth_Real_v_predBy_NN1thick
0,102496,54.785907,-110.129320,00/12-08-067-01W4/0,14000,13000,561.0,475.0,1,3,"[{'UWI': '00/09-11-067-02W4/0', 'neighbor': 1,...",544.0,630.0,86.0,475.0,0.0
1,102497,54.782284,-110.269446,00/07-08-067-02W4/0,14000,13000,604.5,515.0,1,3,"[{'UWI': '00/10-29-067-02W4/0', 'neighbor': 1,...",529.0,613.0,84.0,520.5,-5.5
2,102498,54.785892,-110.186851,00/09-11-067-02W4/0,14000,13000,564.0,480.0,1,3,"[{'UWI': '00/12-08-067-01W4/0', 'neighbor': 1,...",529.0,613.0,84.0,480.0,0.0
3,102500,54.829624,-110.269422,00/10-29-067-02W4/0,14000,13000,636.5,549.0,1,3,"[{'UWI': '00/10-08-068-02W4/0', 'neighbor': 1,...",529.0,613.0,84.0,552.5,-3.5
5,102503,54.771449,-110.402983,00/11-04-067-03W4/0,14000,13000,553.5,488.5,1,2,"[{'UWI': '00/10-08-067-03W4/0', 'neighbor': 1,...",493.5,558.0,64.5,489.0,-0.5


#### Save dataframe as hdf

In [ ]:
#### Save dataframe as hdf

# load_dir = output_data_inst.base_path_for_all_results+ "/" + output_data_inst.path_wellsKNN
# load_filename = output_data_inst.wellsKNN_results_wells_df+output_data_inst.default_results_file_format
# load_results_full_file_path = load_dir+"/"+load_filename

# df_completed_trainAndTest.to_hdf(load_results_full_file_path, key='df', mode='w')